## Read in separate raw data files and create files of individual raster scans
### A. Ordog, June 2022
### NOTE: please do not modify
#### July 07, 2022: added in/out directories
#### July 12, 2022: added option of more than one raster scan for evening and morning
#### Aug 15, 2022: removed old code no longer needed, included check of elevation range during scans

## Import packages and define survey day and file directory

In [ ]:
import dva_sdhdf_combine_v3
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from astropy.modeling import models, fitting

day ='38'
Nm = 1 # number of morning raster scans
Ne = 1 # number of evening raster scans

#### Change the directory to where the files are located" ####
dir_in = '/home2/DATA_AO/DVA_DATA/survey_phase3_day'+day+'/'
#dir_out = '/media/ordoga/DVA_data/survey_phase3_day'+day+'/'
dir_out = '/media/ordoga/15m_band1_survey/dva_phase3/survey_phase3_day'+day+'/'
##############################################################

## Read in the file listing raster scan start and stop times:

In [ ]:
raster1_start = []
raster2_start = []
raster1_stop = []
raster2_stop = []
RA1 = []
RA2 = []
DEC1 = []
DEC2 = []

rast_list = ['a','b','c']

for i in range(0,Ne):
    
    if Ne == 1:
        rast1name = 'dva_survey_phase3_day0'+day+'_raster1.txt'
    else:
        rast1name = 'dva_survey_phase3_day0'+day+'_raster1'+rast_list[i]+'.txt'
        
    with open(dir_out+rast1name) as fp:
        for line in fp:  
            raster1_start.append(line.split()[3])
            raster1_stop.append(line.split()[4])
            RA1.append(float(line.split()[1])*360/24)
            DEC1.append(float(line.split()[2]))
            
for i in range(0,Nm):
    
    if Nm == 1:
        rast2name = 'dva_survey_phase1_day0'+day+'_raster2.txt'
    else:
        rast2name = 'dva_survey_phase1_day0'+day+'_raster2'+rast_list[i]+'.txt'
        
    with open(dir_out+rast2name) as fp:
        for line in fp:  
            raster2_start.append(line.split()[3])
            raster2_stop.append(line.split()[4])
            RA2.append(float(line.split()[1])*360/24)
            DEC2.append(float(line.split()[2]))

print('Evening raster starts:')
print(raster1_start)
print('Evening raster stops:')
print(raster1_stop)
print('')
print('Morning raster starts:')
print(raster2_start)
print('Morning raster stops:')
print(raster2_stop)

## Make files for each raster scan

### ***Note: skip this if files already made

In [ ]:
%%time

imp.reload(dva_sdhdf_combine_v3)
fsamp = 2
rast_list = ['a','b','c']

for i in range(0,Ne):
#for i in range(0,1):
    
    if Ne == 1:
        rast1name = 'dva_survey_phase1_day0'+day+'_raster1_median'
    else:
        rast1name = 'dva_survey_phase1_day0'+day+'_raster1'+rast_list[i]
        
    dva_sdhdf_combine_v3.combine(dir_in,dir_out,raster1_start[i],raster1_stop[i],
                                 rast1name,freq_s=fsamp,freq_avg=True)

for i in range(0,Nm):
#for i in range(1,Nm):   
    if Nm == 1:
        rast2name = 'dva_survey_phase1_day0'+day+'_raster2'
    else:
        rast2name = 'dva_survey_phase1_day0'+day+'_raster2'+rast_list[i]
        
    dva_sdhdf_combine_v3.combine(dir_in,dir_out,raster2_start[i],raster2_stop[i],
                                 rast2name,freq_s=fsamp,freq_avg=True)

## Read in raster scan files:

In [ ]:
%%time

rast_list = ['a','b','c']

dec_rast1   = []; dec_rast2   = []
ra_rast1    = []; ra_rast2    = []
el_rast1    = []; el_rast2    = []
az_rast1    = []; az_rast2    = []
t_rast1     = []; t_rast2     = []
noise_rast1 = []; noise_rast2 = []
RR_rast1    = []; RR_rast2    = []
LL_rast1    = []; LL_rast2    = []
reRL_rast1  = []; reRL_rast2  = []
imRL_rast1  = []; imRL_rast2  = []

for i in range(0,Ne):
    
    print(i)
    
    if Ne == 1:
        rast1name = 'dva_survey_phase1_day0'+day+'_raster1.h5'
    else:
        rast1name = 'dva_survey_phase1_day0'+day+'_raster1'+rast_list[i]+'.h5'
        
    file = h5py.File(dir_out+rast1name,'r')
    dataset1 = file['data']['beam_0']['band_SB0']['scan_0'] 
    dec_rast1.append(dataset1['metadata']['declination'])
    ra_rast1.append(dataset1['metadata']['right_ascension'])
    el_rast1.append(dataset1['metadata']['elevation'])
    az_rast1.append(dataset1['metadata']['azimuth'])
    t_rast1.append(dataset1['metadata']['utc'])
    noise_rast1.append(dataset1['metadata']['noise_state'])  
    RR_rast1.append(dataset1['data'][:,0,:])
    LL_rast1.append(dataset1['data'][:,1,:])
    reRL_rast1.append(dataset1['data'][:,2,:])
    imRL_rast1.append(dataset1['data'][:,3,:])
        

for i in range(0,Nm):
    
    print(i)
    
    if Nm == 1:
        rast2name = 'dva_survey_phase1_day0'+day+'_raster2.h5'
    else:
        rast2name = 'dva_survey_phase1_day0'+day+'_raster2'+rast_list[i]+'.h5'
        
    file = h5py.File(dir_out+rast2name,'r')
    dataset2 = file['data']['beam_0']['band_SB0']['scan_0']
    dec_rast2.append(dataset2['metadata']['declination'])
    ra_rast2.append(dataset2['metadata']['right_ascension'])
    el_rast2.append(dataset2['metadata']['elevation'])
    az_rast2.append(dataset2['metadata']['azimuth'])
    t_rast2.append(dataset2['metadata']['utc'])
    noise_rast2.append(dataset2['metadata']['noise_state'])  
    RR_rast2.append(dataset2['data'][:,0,:])
    LL_rast2.append(dataset2['data'][:,1,:])
    reRL_rast2.append(dataset2['data'][:,2,:])
    imRL_rast2.append(dataset2['data'][:,3,:])
    
freq_rast = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6


In [ ]:
ra_rast1_deg = []
t_plt_rast1 = []

ra_rast2_deg = []
t_plt_rast2 = []

for i in range(0,Ne):
    
    ra_rast1_deg.append(ra_rast1[i]*360./24.)
    t_plt_rast1.append(Time(t_rast1[i], format='isot',scale='utc').mjd)
    
for i in range(0,Nm):
    
    ra_rast2_deg.append(ra_rast2[i]*360./24.)
    t_plt_rast2.append(Time(t_rast2[i], format='isot',scale='utc').mjd)

print(t_plt_rast1)
print(t_plt_rast2)

## Plot raster scans:

In [ ]:
fs = 12
freq_plt = 650. # in MHz
df = freq_rast[1]-freq_rast[0]
wf = np.where(abs(freq_rast-freq_plt)<df)[0][0]

if (Ne > 1 or Nm > 1):
    fig1, axs = plt.subplots(2,2,figsize=(12,12))
    for i in range(0,2):
        
        t1_plt = Time(raster1_start[i], format='isot',scale='utc').mjd
        t2_plt = Time(raster1_stop[i], format='isot',scale='utc').mjd
        wplt1 = np.where((t_plt_rast1[i] >= t1_plt) & (t_plt_rast1[i] <= t2_plt))[0]
        print(wplt1)
        
        plot1 = axs[i,0].scatter( (ra_rast1_deg[i][wplt1]-RA1[i])*np.cos(dec_rast1[i][wplt1]*np.pi/180.), dec_rast1[i][wplt1],
                                s=20,c=10.*np.log10(LL_rast1[i][wplt1,wf]),cmap='viridis',vmin=72, vmax=74)
        axs[i,0].scatter(0,DEC1[i],s=30,color='black')
        
        
        t1_plt = Time(raster2_start[i], format='isot',scale='utc').mjd
        t2_plt = Time(raster2_stop[i], format='isot',scale='utc').mjd
        wplt2 = np.where((t_plt_rast2[i] >= t1_plt) & (t_plt_rast2[i] <= t2_plt))[0]

        plot2 = axs[i,1].scatter( (ra_rast2_deg[i][wplt2]-RA2[i])*np.cos(dec_rast2[i][wplt2]*np.pi/180.), dec_rast2[i][wplt2],
                                s=20,c=10.*np.log10(LL_rast2[i][wplt2,wf]),cmap='viridis',vmin=72, vmax=74)
        axs[i,1].scatter(0,DEC2[i],s=30,color='black')
        
        for j in range(0,2):
            axs[i,j].set_xlabel('$\Delta$ RAcos($\delta$) (deg.)',fontsize=fs)
            axs[i,j].set_ylabel('Dec. (deg.)',fontsize=fs)
            axs[i,j].tick_params(axis="x", labelsize=fs)
            axs[i,j].tick_params(axis="y", labelsize=fs)
            axs[i,j].set_xlim(5,-5)
            axs[i,j].set(aspect='equal')
            axs[i,j].grid()
    
        
else:
    fig1, axs = plt.subplots(1,2,figsize=(12,6))    
    t1_plt = Time(raster1_start[0], format='isot',scale='utc').mjd
    t2_plt = Time(raster1_stop[0], format='isot',scale='utc').mjd
    wplt1 = np.where((t_plt_rast1[0] >= t1_plt) & (t_plt_rast1[0] <= t2_plt))
    t1_plt = Time(raster2_start[0], format='isot',scale='utc').mjd
    t2_plt = Time(raster2_stop[0], format='isot',scale='utc').mjd
    wplt2 = np.where((t_plt_rast2[0] >= t1_plt) & (t_plt_rast2[0] <= t2_plt))
    dB1 = 68
    dB2 = 74
    plot1 = axs[0].scatter( (ra_rast1_deg[0][wplt1]-RA1[0])*np.cos(dec_rast1[0][wplt1]*np.pi/180.), dec_rast1[0][wplt1],
                            s=20,c=10.*np.log10(LL_rast1[0][wplt1,wf]),cmap='viridis',vmin=72, vmax=74)
    axs[0].scatter(0,DEC1[0],s=30,color='black')

    plot2 = axs[1].scatter( (ra_rast2_deg[0][wplt2]-RA2[0])*np.cos(dec_rast2[0][wplt2]*np.pi/180.), dec_rast2[0][wplt2],
            s=20,c=10.*np.log10(LL_rast2[0][wplt2,wf]),cmap='viridis',vmin=72, vmax=74)
    axs[1].scatter(0,DEC2,s=30,color='black')

    for i in range(0,2):
        axs[i].set_xlabel('$\Delta$ RAcos($\delta$) (deg.)',fontsize=fs)
        axs[i].set_ylabel('Dec. (deg.)',fontsize=fs)
        axs[i].tick_params(axis="x", labelsize=fs)
        axs[i].tick_params(axis="y", labelsize=fs)
        axs[i].set_xlim(5,-5)
        axs[i].set(aspect='equal')
        axs[i].grid()
    
#plt.savefig('../DVA2/PLOTS/rasters_day45.jpg')

## Check elevation range of each raster scan:

In [ ]:
ii = 2900
print(freq_rast[ii])
plt.plot(t_plt_rast2[0],el_rast2[0])
#plt.plot(t_plt_rast1[0],10*np.log10(LL_rast1[0][:,ii]))
#plt.ylim(70,90)
plt.grid()

# Further binning frequency channels for select scan

In [ ]:
def freq_avg(step,i0):

    RR_avg = np.empty([len(t_set),int(len(freq[i0:-1])/step)])
    LL_avg = np.empty([len(t_set),int(len(freq[i0:-1])/step)])
    reRL_avg = np.empty([len(t_set),int(len(freq[i0:-1])/step)])
    imRL_avg = np.empty([len(t_set),int(len(freq[i0:-1])/step)])
    freq_avg = np.empty([int(len(freq[i0:-1])/step)])

    for i in range(0,int(len(freq[i0:-1])/step)-1):
    
        i1 = i0+i*step
        i2 = i0+(i+1)*step
    
        freq_avg[i] = np.nanmean(freq[i1:i2])
        #print(freq[i1:i2],freq_avg[i])
    
        RR_avg[:,i] = np.nanmedian(RR_set[:,i1:i2],axis=1)
        LL_avg[:,i] = np.nanmedian(LL_set[:,i1:i2],axis=1)
        reRL_avg[:,i] = np.nanmedian(reRL_set[:,i1:i2],axis=1)
        imRL_avg[:,i] = np.nanmedian(imRL_set[:,i1:i2],axis=1)
    
    return RR_avg, LL_avg, reRL_avg, imRL_avg, freq_avg


In [ ]:
%%time

file = h5py.File('/media/ordoga/15m_band1_survey/dva_phase3/survey_phase3_day54/dva_survey_phase3_day054_raster1.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6

dataset = file['data']['beam_0']['band_SB0']['scan_0']

dec_set = dataset['metadata']['declination']
ra_set = dataset['metadata']['right_ascension']
el_set =dataset['metadata']['elevation']
az_set = dataset['metadata']['azimuth']
t_set = dataset['metadata']['utc']
noise_set = dataset['metadata']['noise_state']
int_time = dataset['metadata']['integration_time']
corrupt = dataset['metadata']['corrupted']
trim_flag = dataset['metadata']['trim_scan_flag']

# Add the spectrometer power data to the corresponding arrays:
RR_set = dataset['data'][:,0,:]
LL_set = dataset['data'][:,1,:]
reRL_set = dataset['data'][:,2,:]
imRL_set = dataset['data'][:,3,::]

In [ ]:
step = 12
i0 = 81
RR_avg, LL_avg, reRL_avg, imRL_avg, freq_bin = freq_avg(step,i0)
#print(len(freq_bin))


In [ ]:
outfiles = '/media/ordoga/15m_band1_survey/dva_phase3/survey_phase3_day54/'
file_ex = 'dva_survey_phase3_day054_raster1.h5'
outname = 'dva_survey_phase3_day054_raster1_1MHz_med.h5'

cmd2 = 'cp '+outfiles+file_ex+' '+outfiles+outname

os.system(cmd2)
file2 = h5py.File(outfiles+outname,'r+')

del file2['data']['beam_0']['band_SB0']

# Create band and scan groups:
file2['data']['beam_0'].create_group("band_SB0")
file2['data']['beam_0']['band_SB0'].create_group(f"scan_0")
print('0')

# Create power dataset:
dat = np.empty((len(t_set), 4, len(freq_bin[0:-1])), dtype=float)
file2['data']['beam_0']['band_SB0']['scan_0'].create_dataset("data", data=dat)
file2['data']['beam_0']['band_SB0']['scan_0']['data'][:,0,:] = RR_avg[:,0:-1]
file2['data']['beam_0']['band_SB0']['scan_0']['data'][:,1,:] = LL_avg[:,0:-1]
file2['data']['beam_0']['band_SB0']['scan_0']['data'][:,2,:] = reRL_avg[:,0:-1]
file2['data']['beam_0']['band_SB0']['scan_0']['data'][:,3,:] = imRL_avg[:,0:-1]
print('1')

# Create metadata with timestamps and coordinates:
metadata_content = [t_set,az_set,el_set,ra_set,dec_set,int_time,corrupt,noise_set,trim_flag]  
#print(noise)
col_names = ["utc", "azimuth", "elevation", "right_ascension", "declination", "integration_time",
             "corrupted","noise_state","trim_scan_flag"]
col_types = np.dtype({'names':col_names,'formats':["S32", "f8", "f8", "f8", "f8", "f8", "?", "i8","i8"] } )
rec_arr = np.rec.array(metadata_content,dtype=col_types)        
file2['data']['beam_0']['band_SB0']['scan_0'].create_dataset("metadata",data=rec_arr)
print('2')

# Create frequency dataset:
file2['data']['beam_0']['band_SB0'].create_dataset("frequency",dtype="f8",data=freq_bin[0:-1]*1e6)

# Create polarizations dataset:
pol_labels = [b"ReRR",b"ReLL",b"ReRL",b"ImRL"]
file2['data']['beam_0']['band_SB0'].create_dataset("polarization",dtype="S32",data=pol_labels)

In [ ]:
filetest = h5py.File(outfiles+'dva_survey_phase3_day038_raster1_1MHz_med.h5','r')
freq_test = filetest['data']['beam_0']['band_SB0']['frequency'][:]/1e6

dataset_test = filetest['data']['beam_0']['band_SB0']['scan_0']

dec_test = dataset_test['metadata']['declination']
ra_test = dataset_test['metadata']['right_ascension']
el_test =dataset_test['metadata']['elevation']
az_test = dataset_test['metadata']['azimuth']
t_test = dataset_test['metadata']['utc']
t_test_mjd = Time(t_test, format='isot',scale='utc').mjd

RR_test = dataset_test['data'][:,0,:]
LL_test = dataset_test['data'][:,1,:]
reRL_test = dataset_test['data'][:,2,:]
imRL_test = dataset_test['data'][:,3,::]

fs = 12
freq_plt = 650. # in MHz
df = freq_test[1]-freq_test[0]
wf = np.where(abs(freq_test-freq_plt)<df)[0][0]

fig1, axs = plt.subplots(1,1,figsize=(12,12))

plt.scatter(t_test_mjd,LL_test[:,wf])